In [1]:
!pip install datasets evaluate accelerate

In [2]:
from datasets import load_dataset
ds=load_dataset('imdb',split='train[:50%]')
ds

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 12500
})

In [4]:
data=ds.train_test_split(test_size=0.15,shuffle=True)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10625
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1875
    })
})

In [5]:
data['train'][0]

{'text': "I go to a lot of movies, often I bring my 5 year old son, I am so glad I did not bring him to this one. There are many references to sex and a skinny dipping scene, however, that is not the primary reason I would not take him to it. The trailers lead you to believe it is a light-hearted comedy; nevertheless, virtually all of the funny moments are in the previews. I kept waiting for it to get interesting, funny, or anything but serious; however, I nearly fell asleep as the plot-less story dragged on. I understand that dogs can be great company, that being said, the entire story focused on a poorly behaving dog that the owners were not savvy enough to train. If a human caused this much damage and mayhem that person would be banned. The worst movie I've ever seen with Jenifer Aniston or Owen Wilson, a waste of their talent. The best way to sum up this movie is, couple gets unruly dog, couple falls in love with dog, dog dies, couple sad. The End.",
 'label': 0}

In [6]:
data['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [7]:
num_labels=2
label2id={'neg':0,'pos':1}
id2label={0:'neg',1:'pos'}


In [8]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
model_id='distilbert/distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model=AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=num_labels,label2id=label2id,id2label=id2label)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

# Emotion

In [2]:
from datasets import load_dataset
emotion_db=load_dataset('dair-ai/emotion',split='train[:50%]')
emotion_db=emotion_db.train_test_split(test_size=0.15,shuffle=True)
emotion_db

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6800
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [10]:
emotion_db['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [3]:
names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
names

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [4]:
emo_id2label={idx:x for idx,x in enumerate(names)}
emo_id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [5]:
emo_label2id={x:idx for idx,x in emo_id2label.items()}
emo_label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [8]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
model_id='distilbert/distilbert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(model_id)
emo_num_labels=len(names)
emo_model=AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=emo_num_labels,label2id=emo_label2id,id2label=emo_id2label,ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def token_function(data):
  return tokenizer(data['text'],truncation=True,padding='max_length',max_length=270,return_tensors='pt')

train_token=emotion_db['train'].map(token_function,batched=True)
test_token=emotion_db['test'].map(token_function,batched=True)

Map:   0%|          | 0/6800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [19]:
train_token

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 6800
})

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate
import numpy as np
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [22]:
from huggingface_hub import login
login()

In [14]:
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(
    output_dir="./emo_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)
trainer = Trainer(
    model=emo_model,
    args=training_args,
    train_dataset=train_token,
    eval_dataset=test_token,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.406968,0.882500


TrainOutput(global_step=425, training_loss=0.6508951344209559, metrics={'train_runtime': 187.2937, 'train_samples_per_second': 36.307, 'train_steps_per_second': 2.269, 'total_flos': 475053697584000.0, 'train_loss': 0.6508951344209559, 'epoch': 1.0})

In [36]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [16]:
emo_model.save_pretrained('./emo_model1')
tokenizer.save_pretrained("./emo_model1")

('./emo_model1/tokenizer_config.json',
 './emo_model1/special_tokens_map.json',
 './emo_model1/vocab.txt',
 './emo_model1/added_tokens.json',
 './emo_model1/tokenizer.json')

In [20]:
train_token[0]['text']

'i should be feeling eager to leap into stash of fabric and make something'

In [21]:
from transformers import pipeline
pipe=pipeline('text-classification',model='nnhwin/emo_model')
output=pipe('i should be feeling eager to leap into stash of fabric and make something')
output

config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'joy', 'score': 0.9168117046356201}]

In [1]:
import gradio as gr
from transformers import pipeline

# Initialize the pipeline
pipe = pipeline('text-classification', model='nnhwin/emo_model')

# Function to classify text
def classify_text(input_text):
    output = pipe(input_text)
    return output

# Create Gradio interface
interface = gr.Interface(
    fn=classify_text,          # Function to call
    inputs="text",             # Input type (text box)
    outputs="json",            # Output type (JSON for displaying result)
    title="Emotion Classifier",  # Title of the Gradio app
    description="Classify emotions using the nnhwin/emo_model.",  # Description
)

# Launch the Gradio app
interface.launch()


/usr/local/lib/python3.10/dist-packages/starlette/formparsers.py:12: FutureWarning: Please use `import python_multipart` instead.
  import multipart
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3d009e8ab964cd783.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1